# Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial - Inteligencia Artificial para Juegos (Game AI) - Sesión 3 (Ejemplo y Tarea)

<font color='orange'>Entorno de *k en raya* (TicTacToe, cuando $k=3$) para búsqueda adversarial en un juego de suma constante</font>

El presente notebook aborda el problema de busqueda adversarial en el juego de k-en-raya (generalizacón del 3 en raya pero en tableros de $h$ filas por $v$ columnas y gana el jugador que haga primero una raya de tamaño $k$, uniendo $k$ piezas adyacentes en forma horizontal, vertical o diagonal).


La mayor parte del entorno de juego (clase TicTacToe) esta implementada, solo esta faltando implementar la funcion que calcula la utilidad de un nuevo estado (funcion <b>compute_utility</b>). Está implementado también el algoritmo MIN-MAX que puede decidir movidas en el juego implementado.

Recordar que en el algoritmo `minimax`, los valores altos de utilidad son buenos para el jugador MAX.

Considerar que las fichas del jugador MAX son **X**, mientas que las fichas del jugador MIN son **O**.

### TAREA:
Modificar la función `compute_utility`, de la clase `TicTacToe`, para que funcione para el caso general ($k>1$). Se habilitará una carpeta en el campus virtual, para que suban sus notebooks resueltos.

### Clase <b>Game</b>

Esta es una clase genérica para definir un entorno de juego. Es parecida a la clase `Problem` de búsqueda, pero en vez del método que devuelve el costo de camino se tiene un metodo que devuelve la utilidad de un jugador en un estado dado. También la funcion test de objetivo es reemplazada por un test de estado terminal (`terminal_test`). Para crear una clase de un juego específico se debe hacer una subclase de `Game` e implementar los métodos actions, `result`, `utility`, y `terminal_test`. El atributo `.initial` (estado inicial del juego) deberá ser inicializado en el constructor de la clase concreta. No editar esta clase `Game`.

In [0]:
class Game:

    def actions(self, state):
        """Retorna una lista de movidas permitidas en el estado actual state."""
        raise NotImplementedError

    def result(self, state, move):
        """Retorna el nuevo estado que resulta de hacer una movida move en el estado state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Retorna el valor de utilidad para el jugador player en el estado terminal state."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Retorna True si el estado state es un estado terminal del juego."""
        return not self.actions(state)

    def to_move(self, state):
        """Retorna el jugador que le toca jugar en el presente estado state."""
        return state.to_move

    def display(self, state):
        """Imprime o displaya el state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players, verbose):
        """Controlador del juego:
        Llama alternadamente a cada jugador pasandole el estado actual del juego y ejecutando la movida retornada."""
        state = self.initial
        numJugada = 0
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                numJugada = numJugada + 1
                if verbose:
                  print("Jugada", numJugada, ": Turno del jugador", player.__name__)
                  self.display(state)
                  print("*************************************************")
                if self.terminal_test(state):
                    print("Jugada", numJugada, "(final): Turno del jugador", player.__name__)
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial)) #retorna utilidad del 1er jugador al acabar el juego

### Clase <b>TicTacToe: Implementado únicamente para 3 en raya</b>

Esta es una subclase de `Game` para definir el entorno del juego k en Raya (generalizacion de `TicTacToe`). Las dimensiones del tablero son definidas en el constructor (usando los argumentos: h=número de filas, v=número de columnas, k=número de elementos en raya para ganar). Primer jugador (Max) es 'X' y el otro jugador (Min) es 'O'. Un estado en este juego es una tupla (`GameState`) con los siguientes campos:
 - to_move: almacena el jugador que le toca jugar 
 - utility: almacena la utilidad del estado
 - board: almacena las posiciones ocupadas en el tablero en la forma de un dicccionario de entradas {(x, y): Player}, donde Player puede ser 'X' o 'O'
 - moves: almacena las movidas posibles a partir del estado en la forma de una lista de tuplas que representan posiciones (x, y) 

In [0]:
from collections import namedtuple
GameState = namedtuple('GameState', 'to_move, utility, board, moves') #Un estado es una tupla con nombres de campos (namedtuple)
import random
import itertools
import copy

class TicTacToe(Game):
    
    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k

        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """Movidas legales son todas las posiciones aun sin marcar (el estado almacena las movidas legales)"""
        return state.moves

    def result(self, state, move):
        """Retorna el nuevo estado de hacer la movida move en el estado state ."""
        if move not in state.moves:
            return state  # Si es una movida ilegal retorna sin cambiar el estado
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Retorna la utilidad del player en estado terminal state; 1 si ganó, -1 si perdió, 0 empate."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """Un estado es terminal si hay un ganador o no hay mas movidas posibles."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """Retorna  1 si player='X'  ha llegado a estado terminal ganador con movida move, 
           Retorna -1 si player='O' ha llegado a estado terminal ganador con movida move; 
           Retornas 0 en cualquier otro caso"""
        #TODO
        #Cálculo de utilidad para el caso k=3. Modificar para el caso general (k>1)
        xa, ya = move
        # Horizontal
        last_mark = board.get((xa,ya))
        
        if (last_mark == 'X'):
            for jj in range(1,8+1):
                if (jj == 1):
                    x_step = 1
                    y_step = 0
                elif (jj == 2):
                    x_step = -1
                    y_step = 0
                elif (jj == 3):
                    x_step = 0
                    y_step = 1
                elif (jj == 4):
                    x_step = 0
                    y_step = -1
                elif (jj == 5):
                    x_step = 1
                    y_step = 1
                elif (jj == 6):
                    x_step = -1
                    y_step = -1
                elif (jj == 7):
                    x_step = +1
                    y_step = -1
                elif (jj == 8):
                    x_step = -1
                    y_step = +1
                    
                for ii in range(1,self.k+1):     
                    x_new = xa + x_step
                    y_new = ya + y_step
                
                    if ((x_new < 0)&(self.h < x_new)&(y_new < 0)&(self.v < y_new)):
                        #si te saliste, mira la siguiente forma de buscar k en raya
                        continue
                
                    mark_new = board.get((x_new, y_new))
                
                    if mark_new == 'O':
                    # no gano nadie
                        return 0
                    if ii == self.k:
                        if mark_new == 'X':
                            return 1
                return 0
            
        else:
            
            for jj in range(1,8+1):
                if (jj == 1):
                    x_step = 1
                    y_step = 0
                elif (jj == 2):
                    x_step = -1
                    y_step = 0
                elif (jj == 3):
                    x_step = 0
                    y_step = 1
                elif (jj == 4):
                    x_step = 0
                    y_step = -1
                elif (jj == 5):
                    x_step = 1
                    y_step = 1
                elif (jj == 6):
                    x_step = -1
                    y_step = -1
                elif (jj == 7):
                    x_step = +1
                    y_step = -1
                elif (jj == 8):
                    x_step = -1
                    y_step = +1
                    
                for ii in range(1,self.k+1):     
                    x_new = xa + x_step
                    y_new = ya + y_step
                
                    if ((x_new < 0)&(self.h < x_new)&(y_new < 0)&(self.v < y_new)):
                        #si te saliste, mira la siguiente forma de buscar k en raya
                        continue
                
                    mark_new = board.get((x_new, y_new))
                
                    if mark_new == 'X':
                    # no gano nadie
                        return 0
                    if ii == self.k:
                        if mark_new == 'O':
                            return -1
                return 0

### Algoritmo  <b>MIN-MAX</b>

Este algoritmo escoge una movida para el jugador de turno en un juego dado (game). El algoritmo obtiene recursivamente los valores minimax de los estados sucesores buscando en profundidad en el arbol de juego los estados terminales. De estos estados toma su valor de utilidad para calcular la utilidad de los padres y asi sucesivamente hasta tener la utilidad de todos los sucesores del estado inicial para decidir la movida a ejecutar. 
La implementacion de esta busqueda es a traves de una recursion alternada de las funciones max_value y min_value (una llama a la otra) hasta alcanzar un estado terminal. Cuando la recursion termina todas las movidas tienen una utilidad y se escoje la movida de mayor valor.


In [0]:
argmax = max
infinity = float('inf')

def minimax_decision(state, game):

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minimax_decision:
    return argmax(game.actions(state),
                  key=lambda a: min_value(game.result(state, a)))

### Jugadores </b>

A seguir se implementan 3 agentes jugadores que pueden hacer movidas en un entorno de juego, dado su estado :
- <b>minimax_player</b>:   jugador que hace movidas de acuerdo al algoritmo MIN-MAX
- <b>random_player</b>:    jugador que hace movidas aleatorias (es facil ganarle  ヽ(^o^)ノ )
- <b>human_player</b>:     solicita la movida a un humano


In [0]:
def minimax_player(game, state):
    return minimax_decision(state, game)

def alphabeta_player(game, state):
    return alphabeta_search(state, game)

def random_player(game, state):
    return random.choice(game.actions(state))

def human_player(game, state):
    print("Estado actual:")
    game.display(state)
    print("Movidas disponibles: {}".format(game.actions(state)))
    print("")
    move_string = input('Cuál es tu movida? ')
    try:
        move = eval(move_string)
    except NameError:
        move = move_string
    return move

### Jugando

Crea el juego clasico 3 en raya y llama al controlador de juego. Primer jugador=minimax_player, Segundo jugador=random_player

In [0]:
ttt = TicTacToe(h=3, v=3, k=3)
print(ttt.play_game(minimax_player, random_player, verbose=True))

Jugada 1 : Turno del jugador minimax_player
X . . 
. . . 
. . . 
*************************************************
Jugada 2 : Turno del jugador random_player
X . O 
. . . 
. . . 
*************************************************
Jugada 3 : Turno del jugador minimax_player
X X O 
. . . 
. . . 
*************************************************
Jugada 4 : Turno del jugador random_player
X X O 
O . . 
. . . 
*************************************************
Jugada 5 : Turno del jugador minimax_player
X X O 
O X . 
. . . 
*************************************************
Jugada 6 : Turno del jugador random_player
X X O 
O X O 
. . . 
*************************************************
Jugada 7 : Turno del jugador minimax_player
X X O 
O X O 
X . . 
*************************************************
Jugada 8 : Turno del jugador random_player
X X O 
O X O 
X . O 
*************************************************
Jugada 9 : Turno del jugador minimax_player
X X O 
O X O 
X X O 
*******************

Crea el juego clasico 3 en raya y llama al controlador de juego. Primer jugador=alphabeta_player, Segundo jugador=random_player

In [0]:
ttt = TicTacToe(h=2, v=2, k=3)
print(ttt.play_game(alphabeta_player, random_player, verbose = True))

Crea el juego clasico 3 en raya y llama al controlador de juego. Primer jugador=alphabeta_player, Segundo jugador=human_player

In [0]:
ttt = TicTacToe(h=3, v=3, k=3)
print(ttt.play_game(alphabeta_player, human_player))